# Rolling Regressions

## Imports

http://www.mit.edu/~6.s085/notes/lecture5.pdf

jt -t monokai -f fira -fs 13 -nf ptsans -nfs 11 -N -kl -cursw 5 -cursc r -cellw 95% -T

In [1]:
# <include-rolling_regressions/utils.py>

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# <imports>
import numpy as np
import pandas as pd
import plotly.io as pio
from patsy import dmatrices
import statsmodels.api as sm

from rolling_regressions import utils

pio.renderers.default = "notebook_connected"
pd.options.plotting.backend = "plotly"
pio.templates.default = "seaborn"

## Summary

In this assignment we calculate regression coefficients for daily returns of 210 individual equities versus the market derived from exponentially weighted and boxcar widows of various lengths and compare them to coefficients calculated from a forward boxcar window. Our basis for comparison is that the coefficients calculated from the forward boxcar windows are the "correct" coefficients and we therefore endeavor to determine which set of historical coefficients are most similar. To determine similarity we perform the following analyses across window methodology and length.

* Compare moments
* Compare histograms
* Compare correlations

Returns are calculated from daily adjusted closing prices for the period from 2016-01-01 to 2020-12-31 for 210 equities that meet the criteria from the quantile trading assignment. We establish weights for the exponentially weighted windows based on characteristic times of $\frac{1}{\lambda}$ ranging from 3 to 60 observations (our analysis makes the simplifying assumption that the duration between observations is the same). Historical boxcar windows lengths are calculated as $\frac{2}{\lambda}$ and range from 6 to 120 observations. In order to have one historical boxcar window length with the same length as the forward window, we set the length of the forward boxcar window at 6 observations instead of 5.

The context for most similar is that we are trying to establish coefficients upon which to base hedges with the experiment being that at each point the actual betas are those from the forward window and that we had put in place hedges based on the coefficients from the historical windows. To simulate this, we assume that we establish a long position of \\$10,000 equally weighted across each of the equities, calculate the average $\beta$, and then assume that we had established a position in the market equal to $-\beta \cdot \text{\$10,000}$. 

The subject data set for our analysis includes daily adjusted closing prices for the period from 2016-01-01 to 2020-12-31 for 210 equities that meet the criteria from the quantile trading assignment. We establish weights for the exponentially weighted windows based on characteristic times of $\frac{1}{\lambda}$ ranging from 3 to 60 observations (our analysis makes the simplifying assumption that the duration between observations is the same). Historical boxcar windows lengths are calculated as $\frac{2}{\lambda}$ and range from 6 to 120 observations. In order to have one historical boxcar window length with the same length as the forward window, we set the length of the forward boxcar window at 6 observations instead of 5.

To demonstrate the first main point of the assignment, we use the framework that the forward boxcar window coefficients are the "correct" ones and compare them to the coefficients calculated from the various 

A more macro approach to making the first point can be seen in asking the question, if we were trying to establish betas for purposes of hedging our portfolio of 200 equities, which which windowing method is best? Within the best windowing method, which duration is best across the entire portfolio? Are the different method and duration combinations that are better for certain subsets of the tickers in the portfolio? Do those tickers share any other obvious characteristics? Are the different periods of the entire duration for which different windowing method and duration combinations perform better?

## Analysis

### Load Prices

In [3]:
df_prices = pd.read_csv("df_prices.csv", usecols=["ticker", "date", "adj_close"]).set_index(["ticker", "date"])
excl_tickers = ["AMRC", "AT", "CCO"]
tickers = [s for s in df_prices.index.levels[0][:210].to_list() if s not in excl_tickers]
df_prices = df_prices.loc[tickers  + ["SPY"]].sort_index()
assert df_prices.isna().groupby("ticker").max().sum()[0] == 0

### Compare Weights

The chart below shows the normalized cumulative weight for each of the exponentially weighted window lengths through 120 days. The main point is that observations well past the time $t$ continue to impact the moving average. It general, observations greater than $t$ comprise approximately 35% of the statistics. Given a boxcar window length of $\frac{2}{\lambda}$ for 10 observations of 20, for example ($\lambda = 0.1)$, the equivalent exponentially weighted window affords approximately 13% of its weight to observations greater than 20.

In [19]:
win_lengths = np.array([3, 6, 10, 30, 60, 90])

In [20]:
nobs = 1440
fig = utils.go.Figure()
for w in win_lengths:
    weights = np.power(1 - 1/w, np.arange(nobs))
    weights = weights / weights.sum()
    fig.add_scatter(x=(np.arange(nobs) + 1)[:120], y=pd.Series(weights).cumsum().values, name=f"w = {w}")
fig.update_layout(title_text="Cumulative Weight by Window Length")
fig.show()

### Calculate Returns

In [9]:
df_ret = np.log(df_prices.adj_close.unstack("ticker") / df_prices.adj_close.unstack("ticker").shift())
df_ret

ticker,AAWW,ABM,ACIW,ACM,ADP,ADS,AEGN,AEP,AES,AGCO,...,KMB,KMI,KMX,KO,KR,KRA,KRO,KSS,KSU,SPY
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2016-01-05,-0.010246,0.002346,-0.003392,-0.007058,0.002434,0.008575,-0.009484,0.008195,0.013692,0.014341,...,0.020348,0.020098,-0.004785,0.003531,0.022343,-0.030867,0.000000,0.004631,0.003521,0.001690
2016-01-06,-0.018507,-0.009842,-0.004866,-0.015636,-0.012353,-0.014222,-0.032825,0.003734,-0.035129,-0.009243,...,-0.000465,-0.053390,-0.075910,-0.005420,-0.000951,0.009981,-0.082997,0.005409,-0.047048,-0.012694
2016-01-07,-0.084317,-0.021474,-0.044895,-0.030258,-0.030995,-0.030629,-0.008791,-0.011586,-0.025234,0.000663,...,-0.016883,-0.035841,-0.025153,-0.016679,-0.023581,-0.038600,-0.041223,0.016054,-0.020180,-0.024284
2016-01-08,-0.019964,-0.005253,-0.021661,-0.018173,-0.008285,-0.019344,-0.072666,-0.001544,0.014341,-0.004207,...,-0.012932,0.035841,-0.006602,-0.002646,-0.006596,-0.012334,-0.038820,-0.060379,0.008063,-0.011037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-24,0.006971,-0.012910,0.013588,0.002263,0.007790,0.009768,0.017291,0.008758,-0.009410,-0.004254,...,0.002917,0.000000,-0.009489,0.006759,0.004450,0.024702,-0.000665,-0.020855,0.002557,0.003883
2020-12-28,-0.023611,0.019433,-0.008078,-0.009291,0.000283,0.006368,-0.002601,-0.001721,0.001288,-0.010030,...,0.000448,-0.013833,-0.014185,0.013383,-0.004450,-0.007677,0.001329,0.030873,0.006786,0.008554
2020-12-29,-0.015253,-0.019168,-0.009860,0.005791,-0.009901,-0.020493,-0.021053,-0.001231,0.000429,-0.009044,...,0.001343,-0.005145,0.013534,-0.000554,0.002228,-0.011441,-0.018091,-0.005999,-0.006486,-0.001910


Here is our familiar pattern of generally upward drift with a significant drop and increased volatility at the onset of the pandemic. It's interesting that the performance of this particular set of equities was so much worse than the market over this period.

In [18]:
utils.make_return_chart(df_ret)

### Calculate Betas

Here we calculate betas for each equity for each observation for each window type for each window length (1.3 million in total). We do this by using the rolling covariance matrix function in pandas to get the covariance versus SPY and the variance for each ticker and then calculate betas directly as $\hat{\beta} = \frac{\mathrm{Cov}(x,y)}{\mathrm{Var}(x)}$.

We do a quick check of the the mean $\hat{\beta}$ for the forward boxcar window to make sure data is available for each of our selected tickers.

In [21]:
df_betas = utils.get_betas(df_ret, win_lengths)
fig = df_betas.boxcar_fwd.beta_1.t_03.groupby("ticker").mean().plot(kind="bar", title="Mean Beta")
fig.update_layout(showlegend=False)
fig.show()

  0%|          | 0/6 [00:00<?, ?it/s]

### Compare Moments

This is a dense chart, but allows us to compare the betas for the various combinations of window type and window length from one chart with y_axes for each statistic on the same scale. Note that the window length for all of the length labels for the forward box car window is 6 to facilitate comparison with the other window methods at varying lengths.

* Variance decrease significantly with window length, as we would expect given the larger number of observations included in the longer windows.
* Mean beta decreases as window length increases as a result of covariance decreasing at a faster rate than the variance in the denominator (variance of equity and market decrease as window lenght increases, covariance in the numerator is akin to the product of the variances versus just the variance by itself in the denominator). What's interesting is that the mean beta actually starts to increase again for the window lengths greater than 30. I'm not sure why this would be since the variance is still decreasing.
* Kurtosis decreases with window length similarly to variance.
* By design, the statistics for the boxcar and forward boxcar windows of length 6 (t_03) are nearly identical (but for the lag of 6 days).
* For the shorter windows, the mean beta for the exponentially weighted windows is less than it is for the boxcar windows, likely related to the effectively greater number of observations included in the exponentially weighted windows and the relative impact on covariance versus variance.


In [22]:
df_moments = utils.get_moments(df_betas, start_date="2016-02-01")
fig = utils.make_moments_chart(df_moments)
fig.show()

In [158]:
df_t = df_betas[[("boxcar", "beta_1", "t_03"), ("boxcar_fwd", "beta_1", "t_03")]].loc[df_betas.index.get_level_values("ticker").isin(["AAWW"])]
# df_t.columns = ["boxcar", "boxcar_fwd"]
df_t.head(18)

,win_type,boxcar,boxcar_fwd
,stat,beta_1,beta_1
,win_length,t_03,t_03
date,ticker,,
2016-01-04,AAWW,NaN,0.243053
2016-01-05,AAWW,NaN,0.264163
2016-01-06,AAWW,NaN,0.262276
2016-01-07,AAWW,NaN,0.389213
2016-01-08,AAWW,NaN,0.428037
2016-01-11,AAWW,NaN,0.477142
2016-01-12,AAWW,0.276649,0.324743


In [159]:
t_var = df_ret[["AAWW", "SPY"]].rolling(6).cov()["AAWW"].xs("AAWW", level=1)
t_cov = df_ret[["AAWW", "SPY"]].rolling(6).cov()["AAWW"].xs("SPY", level=1)
t_beta = t_cov / t_var
t_beta.head(18)

date
2016-01-04         NaN
2016-01-05         NaN
2016-01-06         NaN
2016-01-07         NaN
2016-01-08         NaN
2016-01-11         NaN
2016-01-12    0.276649
2016-01-13    0.252613
2016-01-14    0.341983
2016-01-15    0.435454
2016-01-19    0.267318
2016-01-20    0.344896
2016-01-21    0.297336
2016-01-22    0.022010
2016-01-25    0.108743
2016-01-26    0.091141
2016-01-27    0.162745
2016-01-28    0.337533
Name: AAWW, dtype: float64

In [160]:
t_var = df_ret[["AAWW", "SPY"]].rolling(6).cov().groupby("ticker").shift(-6)["AAWW"].xs("AAWW", level=1)
t_cov = df_ret[["AAWW", "SPY"]].rolling(6).cov().groupby("ticker").shift(-6)["AAWW"].xs("SPY", level=1)
t_beta = t_cov / t_var
t_beta.head(18)

date
2016-01-04    0.276649
2016-01-05    0.252613
2016-01-06    0.341983
2016-01-07    0.435454
2016-01-08    0.267318
2016-01-11    0.344896
2016-01-12    0.297336
2016-01-13    0.022010
2016-01-14    0.108743
2016-01-15    0.091141
2016-01-19    0.162745
2016-01-20    0.337533
2016-01-21    0.319036
2016-01-22    0.334491
2016-01-25    0.367169
2016-01-26    0.352239
2016-01-27    0.331858
2016-01-28    0.342980
Name: AAWW, dtype: float64

In [14]:
moments = []
for time in times:
    df_select = df_betas[[("exp_wm", "beta_1", f"t_{time:02d}"), ("boxcar", "beta_1", f"t_{time:02d}"), ("boxcar_fwd", "beta_1", f"t_{time:02d}")]].dropna()
#     fig = utils.px.histogram(df_select.stack(["win_type", "stat"]).reset_index(), x=f"t_{time:02d}", color="win_type", barmode="overlay", title=f"Histogram of Coefficients: t_{time:02d}", histnorm="percent", marginal="box", height=600, opacity=0.7)
#     fig.show()
    print(df_select.describe())

win_type         exp_wm         boxcar     boxcar_fwd
stat             beta_1         beta_1         beta_1
time               t_03           t_03           t_03
count     258336.000000  258336.000000  258336.000000
mean           0.238815       0.244124       0.245302
std            0.296890       0.331529       0.380557
min           -5.389845      -8.037017     -10.692852
25%            0.048437       0.043796       0.034279
50%            0.179555       0.183610       0.183568
75%            0.378450       0.394936       0.408231
max            7.392998       8.306604      10.741401
win_type         exp_wm         boxcar     boxcar_fwd
stat             beta_1         beta_1         beta_1
time               t_06           t_06           t_06
count     257094.000000  257094.000000  257094.000000
mean           0.228918       0.233304       0.244180
std            0.232160       0.244716       0.380077
min           -2.308929      -2.619839     -10.692852
25%            0.066534     

In [ ]:
time = times[0]
for time in times:
    df_select = df_betas[[("exp_wm", "beta_1", f"t_{time:02d}"), ("boxcar", "beta_1", f"t_{time:02d}"), ("boxcar_fwd", "beta_1", f"t_{time:02d}")]].dropna()
    fig = utils.px.histogram(df_select.stack(["win_type", "stat"]).reset_index(), x=f"t_{time:02d}", color="win_type", barmode="overlay", title=f"Histogram of Coefficients: t_{time:02d}", histnorm="percent", marginal="box", height=600, opacity=0.7)
    fig.show()
    print(df_select.describe())

In [ ]:
df_corr = df_betas.swaplevel("stat", "win_type", axis=1).beta_1.corr()
flat_index = df_corr.index.to_flat_index().map(lambda x: f"{x[0]}_{x[1]}")
df_corr.index = flat_index
df_corr.columns = flat_index
incl_cols = [f for f in flat_index if not ("boxcar_fwd" in f and int(f[-2:]) != 3)]
df_corr = df_corr.loc[incl_cols, incl_cols]
utils.px.imshow(df_corr, height=600, title="Beta Correlation Matrix")

In [ ]:
print(df_betas[[("boxcar", "beta_1", f"t_{time:02d}"), ("boxcar_fwd", "beta_1", "t_05"), ("exp_wm", "beta_1", f"t_{time:02d}")]].loc[ticker].dropna().describe())

As a check to makes sure the that the calculations appear to be being performed correctly we can plot one ticker, SPY and the variance and covariance. This looks like its doing the right thing - the 90 day moving average with much more muted responses to changes in AAPL variance.

In [ ]:
fig = utils.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_scatter(x=df_betas.index.levels[1], y=df_prices.loc["AAPL"].adj_close, name="AAPL")
fig.add_scatter(x=df_betas.index.levels[1], y=df_betas.loc["AAPL"].exp_wm.var_x.t_05, name="t_05", secondary_y=True)
fig.add_scatter(x=df_betas.index.levels[1], y=df_betas.loc["AAPL"].exp_wm.var_x.t_90, name="t_90", secondary_y=True)
fig.update_layout(title="EWM Variance", showlegend=True)

Covariance appears to be working as well.

In [ ]:
fig = utils.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_scatter(x=df_betas.index.levels[1], y=df_prices.loc["AAPL"].adj_close / df_prices.loc["AAPL"].adj_close.iloc[0] * 100, name="AAPL") 
fig.add_scatter(x=df_betas.index.levels[1], y=df_prices.loc["SPY"].adj_close / df_prices.loc["SPY"].adj_close.iloc[0] * 100, name="SPY", line=dict(color=utils.COLORS[3]))
fig.add_scatter(x=df_betas.index.levels[1], y=df_betas.loc["AAPL"].cov_xy.t_05, name="cov_xy.t_05", secondary_y=True, line=dict(color=utils.COLORS[1]))
fig.add_scatter(x=df_betas.index.levels[1], y=df_betas.loc["AAPL"].cov_xy.t_90, name="cov_xy.t_90", secondary_y=True, line=dict(color=utils.COLORS[2]))
fig.update_layout(title="EWM Covariance", showlegend=True)

This can be simplified with the calculation of ewma above by just calcing the var and covs and the rest is the same.

In [ ]:
col_list = []
for t in times:
    s_var = pd.Series((np.concatenate(np.repeat(np.expand_dims(np.eye(len(df_ret.columns)), axis=0), len(df_ret.index), axis=0), axis=0) * df_ret.rolling(window=2 * t).cov()).sum(axis=1), name=("var_x", f"t_{t:02d}"))
    s_cov = df_ret.rolling(window=2 * t).cov()["SPY"]
    s_cov.name = ("cov_xy", f"t_{t:02d}")
    col_list.extend([s_var, s_cov])
df_vars = pd.concat(col_list, axis=1)
df_vars = df_vars.loc[df_vars.index.get_level_values("ticker") != "SPY"]
df_vars.columns.names = ["stat", "time"]

df_beta = df_vars["cov_xy"].divide(df_vars["var_x"])
df_beta.columns = pd.MultiIndex.from_tuples([("beta_1", c) for c in df_beta.columns], names=["stat", "time"])

df_betas = pd.concat([df_vars, df_beta], axis=1)
df_betas = df_betas.sort_index(axis=1)
df_betas.index = df_betas.index.swaplevel()
df_betas = df_betas.sort_index()
df_betas.loc["AAPL"].head(60)

### Forward Looking 5 Box Car Coefficients

In [ ]:
df_ret.loc["2020-04-27":"2020-05-11"]

In [ ]:
res = sm.OLS.from_formula(y, X)
res.summary()

In [ ]:
df_cov = df_ret.loc["2020-04-27":"2020-05-11"].cov()
df_cov

In [ ]:
beta = df_cov.SPY.iloc[1] / df_cov.SPY.iloc[0]
beta

In [ ]:
len(df_ret.loc["2020-04-27":"2020-05-11"])

In [ ]:
df_ret

In [ ]:
cov_SPY_SUN = (df_ret.loc["2020-04-27":"2020-05-11"] - df_ret.loc["2020-04-27":"2020-05-11"].mean()).prod(axis=1).sum() / 10
cov_SPY_SUN

In [ ]:
var_SPY_SUN = (df_ret.loc["2020-04-27":"2020-05-11"] - df_ret.loc["2020-04-27":"2020-05-11"].mean()).pow(2).sum() / 10
var_SPY_SUN

In [ ]:
df_ret.loc["2020-04-27":"2020-05-11"].mean().SUN - beta * df_ret.loc["2020-04-27":"2020-05-11"].mean().SPY

## Check to see if